In [60]:
import pandas as pd

import numpy as np

In [2]:
dico_annee = {"2023_2024" : ["Auxerre", "Angers", "Saint-Étienne", "Rodez", "Paris FC"],
              "2022_2023" : ["Le Havre", "Metz", "Bordeaux", "Bastia", "Caen"],
              "2021_2022" : ["Toulouse", "AC Ajaccio", "Auxerre", "Paris FC", "Sochaux"],
              "2020_2021" : ["Troyes", "Clermont Foot", "Toulouse", "Grenoble Foot", "Paris FC"]}

annee = "2023_2024"

In [3]:
event_import = pd.read_json(f"../../../Data_file/Heatmap SB/{annee}.json")

In [163]:
event = event_import[~event_import.location.isna()]

event.sort_values(by = ["match_id", "index"], inplace = True)

event.set_index(["match_id", "period", "possession"], inplace = True)

shot = event[(event.type == "Shot")]

deb_action = event.groupby(level = [0, 1, 2]).head(1).loc[shot.index]

deb_action["type_action"] = deb_action.pass_type.fillna(deb_action.shot_type).fillna(deb_action.type)

loc_deb_action = pd.DataFrame(deb_action.location.tolist(), index = deb_action.index)

df = pd.concat([loc_deb_action, shot.team, deb_action.type_action, shot.shot_outcome == "Goal"], axis = 1)

df = df.reset_index().drop([2, "period", "possession"], axis = 1)

df.replace(to_replace = ['Interception', 'Ball Recovery', 'Duel', 'Recovery', 'Pass', 'Goal Keeper', '50/50'], value = "Open play", inplace = True)

df.columns = ["match_id", "x", "y", "Équipe", "type_action", "But"]

df["Top 5"] = df["Équipe"].isin(dico_annee[annee])

df

C:\Users\fabie\AppData\Local\Temp\ipykernel_21780\400472936.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  event.sort_values(by = ["match_id", "index"], inplace = True)


,match_id,x,y,Équipe,type_action,But,Top 5
0,3896583,120.0,0.1,AC Ajaccio,Corner,False,False
1,3896583,63.7,61.3,Rodez,Open play,False,True
2,3896583,63.7,61.3,Rodez,Open play,True,True
3,3896583,120.0,80.0,AC Ajaccio,Corner,False,False
4,3896583,89.1,8.6,AC Ajaccio,Free Kick,False,False
...,...,...,...,...,...,...,...
8875,3937846,120.0,0.1,Saint-Étienne,Corner,False,True
8876,3937846,5.0,41.4,Rodez,Goal Kick,False,True
8877,3937846,29.2,55.4,Rodez,Open play,False,True
8878,3937846,27.5,26.4,Saint-Étienne,Open play,False,True


In [152]:
df.type_action.unique()

array(['Corner', 'Interception', 'Free Kick', 'Goal Kick',
       'Ball Recovery', 'Duel', 'Recovery', 'Pass', 'Throw-in', 'Penalty',
       'Goal Keeper', '50/50', 'Referee Ball-Drop', 'Kick Off'],
      dtype=object)